입력데이터와 정답데이터(공식코드에서 meta 어쩌고 변수 이름을 쓰는 거)를 묶어서 한 파일로 저장.

입력데이터와 정답데이터에서 `Test_id`의 순서가 같지 않은데 이 둘의 순서 상관 없이 취급하도록 코드가 작성돼있어 결과적으로 정답 데이터가 잘못 된 거나 마찬가지임.

In [1]:
import os
from typing import Callable
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from mymodules.paths import Paths
from mymodules.preprocess import preprocess_A, preprocess_B
from mymodules.feature_engineering import add_features_A, add_features_B


In [2]:
PATH = Paths('open_v2')

In [24]:
if os.path.exists(PATH.aggt_A):
    os.remove(PATH.aggt_A)
if os.path.exists(PATH.aggt_B):
    os.remove(PATH.aggt_B)

In [4]:
train_meta = pd.read_csv(PATH.train_meta, index_col='Test_id')
meta_A = train_meta[train_meta["Test"] == "A"].copy()
meta_B = train_meta[train_meta["Test"] == "B"].copy()

In [26]:
reader = pd.read_csv(PATH.train_A, index_col='Test_id', chunksize=10000)
for i, chunk in enumerate(reader):
    chunk['Label'] = chunk.index.map(meta_A['Label'])
    chunk.to_csv(PATH.aggt_A, mode='a', index='Test_id', header=(i == 0))

In [5]:
reader = pd.read_csv(PATH.train_B, index_col='Test_id', chunksize=10000)
for i, chunk in enumerate(reader):
    chunk['Label'] = chunk.index.map(meta_B['Label'])
    chunk.to_csv(PATH.aggt_B, mode='a', index='Test_id', header=(i == 0))